In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

### KBreport

In [157]:

def get_kbreport(url):
    driver = webdriver.Chrome("/Users/young/Documents/chromedriver")
    driver.get(url)
    old_record = []
    for year in range(1982,2003):

        season_box = driver.find_element(By.CSS_SELECTOR,'#searchFrm > div.ltb-select-left > div.ltb-sl-1 > select.ltb-season-select')
        season_box.send_keys(year)

        #검색
        driver.find_element(By.CSS_SELECTOR,'#searchFrm > div.ltb-select-right > button').click()
        time.sleep(1)
        #행 설정
        row_box = driver.find_element(By.CSS_SELECTOR,'#resultListDiv > div.page-row-box > select')
        row_box.send_keys('100')

        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')

        paging = soup.select('#paging > a')
        last_page = paging[-1]['href']
        print(last_page)
        try:
            last_page = int(re.findall('[0-9]',last_page)[0])
        except:
            last_page = 1

        tf = soup.select('#resultListDiv > table > tbody > tr')

        line = re.split('\n',tf[1].text)
        del line[0],line[-1]
        line

        
        for i in range(1,len(tf)):
            line = re.split('\n',tf[i].text)
            del line[0],line[-1]
            line.insert(0,year)
            old_record.append(line)

    return old_record

In [ ]:
url = "http://www.kbreport.com/history/pitcher/main"
url_basic = 'http://www.kbreport.com/history/pitcher/standard?teamId=&pitcher_type=&year_at=&inning_count='
url_detail = 'http://www.kbreport.com/history/pitcher/advanced?teamId=&pitcher_type=&year_at=&inning_count='
main_record = get_kbreport(url)
basic_record = get_kbreport(url_basic)
detail_record = get_kbreport(url_detail)

In [204]:
main = pd.DataFrame(main_record)
new_main = main[main[0]>=2000]
old_main = main[main[0]<2000]

In [216]:
new_main = new_main.drop([7],axis=1)

main = ['날짜','순위','선수명','팀명','승','패','세','경기','선발','이닝','삼진/9','볼넷/9','홈런/9','BABIP','LOB%','ERA','RA9-WAR','FIP','kFIP','WAR']
old_main = old_main.iloc[:,0:-1]
old_main.columns = main
new_main.columns = main

merged_df = pd.concat([old_main, new_main], axis=0)
merged_df

,날짜,순위,선수명,팀명,승,패,세,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR
0,1982,1,박철순,OB,24,4,7,36,32,224.7,4.33,2.68,0.28,0.213,76.60,1.84,9.52,2.80,2.83,5.97
1,1982,2,황규봉,삼성,15,11,11,47,38,222.3,4.01,1.38,0.45,0.252,70.60,2.47,6.49,2.70,2.68,5.88
2,1982,3,노상수,롯데,14,19,2,44,30,232.3,5.46,2.29,0.70,0.277,72.40,2.94,4.77,2.90,3.03,5.21
3,1982,4,하기룡,MBC,13,10,9,43,29,191.7,5.02,2.49,0.47,0.254,74.50,2.30,5.49,2.86,2.96,4.54
4,1982,5,선우대영,OB,7,6,0,27,8,138.0,4.24,2.02,0.52,0.291,67.60,3.39,2.49,2.96,2.97,3.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2002,96,최향남,LG,7,9,0,23,23,109.0,6.19,4.79,1.32,0.252,71.40,4.46,0.93,5.35,5.30,0.04
1891,2002,97,허진석,한화,0,0,0,2,0,4.0,0.00,0.00,0.00,0.077,100.00,0.00,0.24,3.73,3.00,0.04
1892,2002,98,임봉춘,롯데,0,0,0,3,0,3.7,4.91,2.45,0.00,0.357,71.40,2.45,0.00,3.75,3.55,0.03
1893,2002,99,이명우,롯데,0,0,0,19,0,16.3,6.61,3.31,1.10,0.150,57.40,3.86,0.10,4.44,4.41,0.02


In [223]:
basic = ['날짜','순위','선수명','팀명','경기','선발','완투','완봉','타자','안타','홈런','실점','자책','삼진','볼넷','고4','HBP','폭투','보크','PK']
basic = pd.DataFrame(basic_record,columns=basic)


In [226]:
detail = ['날짜','순위','선수명','팀명','삼진%','볼넷%','삼/볼','피안타율','피출루율','WHIP','BABIP','LOB%','ERA','RA9-WAR','FIP','kFIP','WAR',]
detail = pd.DataFrame(detail_record,columns=detail)

In [234]:
detail

,날짜,순위,선수명,팀명,삼진%,볼넷%,삼/볼,피안타율,피출루율,WHIP,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR
0,1982,1,박철순,OB,12.4,7.7,1.61,0.191,0.254,0.97,0.213,76.60,1.84,9.52,2.80,2.83,5.97
1,1982,2,황규봉,삼성,11.4,3.9,2.91,0.232,0.262,1.00,0.252,70.60,2.47,6.49,2.70,2.68,5.88
2,1982,3,노상수,롯데,14.8,6.2,2.39,0.248,0.294,1.18,0.277,72.40,2.94,4.77,2.90,3.03,5.21
3,1982,4,하기룡,MBC,13.7,6.8,2.02,0.228,0.280,1.11,0.254,74.50,2.30,5.49,2.86,2.96,4.54
4,1982,5,선우대영,OB,11.5,5.5,2.10,0.269,0.307,1.23,0.291,67.60,3.39,2.49,2.96,2.97,3.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2002,96,최향남,LG,15.8,12.2,1.29,0.237,0.330,1.40,0.252,71.40,4.46,0.93,5.35,5.30,0.04
1891,2002,97,허진석,한화,0.0,0.0,0.00,0.077,0.077,0.25,0.077,100.00,0.00,0.24,3.73,3.00,0.04
1892,2002,98,임봉춘,롯데,11.1,5.6,2.00,0.313,0.353,1.64,0.357,71.40,2.45,0.00,3.75,3.55,0.03
1893,2002,99,이명우,롯데,19.7,9.8,2.00,0.151,0.233,0.86,0.150,57.40,3.86,0.10,4.44,4.41,0.02


In [235]:
merged_df = merged_df.merge(basic.iloc[:,4:],left_index=True,right_index=True)
merged_df = merged_df.merge(detail.iloc[:,4:],left_index=True,right_index=True)
merged_df

,날짜,순위,선수명,팀명,승,패,세,경기_x,선발_x,이닝,삼진/9,볼넷/9,홈런/9,BABIP_x,LOB%_x,ERA_x,RA9-WAR_x,FIP_x,kFIP_x,WAR_x,경기_y,선발_y,완투_x,완봉_x,타자_x,안타_x,홈런_x,실점_x,자책_x,삼진_x,볼넷_x,고4_x,HBP_x,폭투_x,보크_x,PK_x,경기,선발,완투_y,완봉_y,타자_y,안타_y,홈런_y,실점_y,자책_y,삼진_y,볼넷_y,고4_y,HBP_y,폭투_y,보크_y,PK_y,삼진%,볼넷%,삼/볼,피안타율,피출루율,WHIP,BABIP_y,LOB%_y,ERA_y,RA9-WAR_y,FIP_y,kFIP_y,WAR_y
0,1982,1,박철순,OB,24,4,7,36,32,224.7,4.33,2.68,0.28,0.213,76.60,1.84,9.52,2.80,2.83,5.97,36,32,15,2,871,151,7,60,46,108,67,1,6,4,0,0,36,32,15,2,871,151,7,60,46,108,67,1,6,4,0,0,12.4,7.7,1.61,0.191,0.254,0.97,0.213,76.60,1.84,9.52,2.80,2.83,5.97
1,1982,2,황규봉,삼성,15,11,11,47,38,222.3,4.01,1.38,0.45,0.252,70.60,2.47,6.49,2.70,2.68,5.88,47,38,8,2,865,188,11,78,61,99,34,3,6,4,1,0,47,38,8,2,865,188,11,78,61,99,34,3,6,4,1,0,11.4,3.9,2.91,0.232,0.262,1.00,0.252,70.60,2.47,6.49,2.70,2.68,5.88
2,1982,3,노상수,롯데,14,19,2,44,30,232.3,5.46,2.29,0.70,0.277,72.40,2.94,4.77,2.90,3.03,5.21,44,30,12,2,954,216,18,97,76,141,59,6,10,4,2,0,44,30,12,2,954,216,18,97,76,141,59,6,10,4,2,0,14.8,6.2,2.39,0.248,0.294,1.18,0.277,72.40,2.94,4.77,2.90,3.03,5.21
3,1982,4,하기룡,MBC,13,10,9,43,29,191.7,5.02,2.49,0.47,0.254,74.50,2.30,5.49,2.86,2.96,4.54,43,29,7,4,781,160,10,68,49,107,53,4,13,6,2,0,43,29,7,4,781,160,10,68,49,107,53,4,13,6,2,0,13.7,6.8,2.02,0.228,0.280,1.11,0.254,74.50,2.30,5.49,2.86,2.96,4.54
4,1982,5,선우대영,OB,7,6,0,27,8,138.0,4.24,2.02,0.52,0.291,67.60,3.39,2.49,2.96,2.97,3.27,27,8,6,1,566,139,8,64,52,65,31,1,4,3,0,0,27,8,6,1,566,139,8,64,52,65,31,1,4,3,0,0,11.5,5.5,2.10,0.269,0.307,1.23,0.291,67.60,3.39,2.49,2.96,2.97,3.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1890,2002,96,최향남,LG,7,9,0,23,23,109.0,6.19,4.79,1.32,0.252,71.40,4.46,0.93,5.35,5.30,0.04,23,23,0,0,476,95,16,62,54,75,58,2,8,3,1,0,23,23,0,0,476,95,16,62,54,75,58,2,8,3,1,0,15.8,12.2,1.29,0.237,0.330,1.40,0.252,71.40,4.46,0.93,5.35,5.30,0.04
1891,2002,97,허진석,한화,0,0,0,2,0,4.0,0.00,0.00,0.00,0.077,100.00,0.00,0.24,3.73,3.00,0.04,2,0,0,0,13,1,0,0,0,0,0,0,0,0,0,0,2,0,0,0,13,1,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.00,0.077,0.077,0.25,0.077,100.00,0.00,0.24,3.73,3.00,0.04
1892,2002,98,임봉춘,롯데,0,0,0,3,0,3.7,4.91,2.45,0.00,0.357,71.40,2.45,0.00,3.75,3.55,0.03,3,0,0,0,18,5,0,2,1,2,1,0,1,0,0,0,3,0,0,0,18,5,0,2,1,2,1,0,1,0,0,0,11.1,5.6,2.00,0.313,0.353,1.64,0.357,71.40,2.45,0.00,3.75,3.55,0.03
1893,2002,99,이명우,롯데,0,0,0,19,0,16.3,6.61,3.31,1.10,0.150,57.40,3.86,0.10,4.44,4.41,0.02,19,0,0,0,61,8,2,8,7,12,6,0,1,1,0,0,19,0,0,0,61,8,2,8,7,12,6,0,1,1,0,0,19.7,9.8,2.00,0.151,0.233,0.86,0.150,57.40,3.86,0.10,4.44,4.41,0.02


In [236]:
merged_df.to_csv('/Users/young/Desktop/merged_kbreport.csv')

---

In [237]:
df_salary_old = pd.read_csv('/Users/young/Downloads/salary_baseball.csv',encoding='cp949')


### Statiz

In [172]:
for i in range(1,2):
    for i in range(1,10):
        s = i = 1
    print(s)

1


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

wait = WebDriverWait(driver, 10)

wait.until(EC.element_to_be_selected(
    (By.CSS_SELECTOR, '#contents > div.sub-content')))


In [87]:
statiz_baseball_salary = pd.DataFrame(columns=["선수","연도","팀","연봉(만원)","WAR"])
for year in range(1997,2022):
    for team in [1,2,3,4,5,6,7,8,9,10,101,102]:
        url = f"http://www.statiz.co.kr/salary.php?opt=0&sopt={year}&cnv=&pos=&te={team}"
        year_salary_data = pd.read_html(url)[1]
        statiz_baseball_salary = pd.concat([statiz_baseball_salary,year_salary_data])

In [88]:
statiz_baseball_salary.to_csv('/Users/young/Desktop/statiz_salary.csv')

In [4]:
statiz_baseball_records = pd.DataFrame(columns=['순', '이름', '팀', 'WAR*', 'G', '타석', '타수', '득점', '안타', '2타', '3타', '홈런',
       '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율',
       '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR*', 'WPA'])
year = 2021
url = f'http://www.statiz.co.kr/stat_at.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=2016&qu=auto&po=51&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
df = pd.read_html(url)[0]
df.columns = df.columns.droplevel(0)
statiz_baseball_records = pd.concat([statiz_baseball_records,df])


### KBO 데이터 수집
---

In [25]:
import time
from selenium.webdriver.chrome.options import Options

In [69]:

url1 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx?sort=ERA_RT"
url2 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic2.aspx?sort=ERA_RT"
url3 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx"
url4 = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail2.aspx"


def seleniums(url):

    options = Options()
    user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
    options.add_argument('user-agent=' + user_agent)

    driver = webdriver.Chrome("/Users/young/Documents/chromedriver")
    driver.get(url)
    
    game_box = driver.find_element(By.CSS_SELECTOR,'#cphContents_cphContents_cphContents_ddlSeries_ddlSeries')
    game_box.send_keys('KBO 정규시즌')

    record_outer = []
    
    for year in range(2002,2023):
        time.sleep(1)
        print(year)

        driver.implicitly_wait(10)
        season_box = driver.find_element(By.CSS_SELECTOR,'#cphContents_cphContents_cphContents_ddlSeason_ddlSeason')
        season_box.send_keys(year)
        time.sleep(1)
        
        path = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[1]/ul/li[2]/a'
        driver.find_element_by_xpath(path).send_keys('ENTER')
        driver.find_element_by_xpath(path).click()
        time.sleep(1)

        next_path = '//*[@id="cphContents_cphContents_cphContents_udpContent"]/div[2]/div[2]/a[2]'
        driver.find_element_by_xpath(next_path).click()
        time.sleep(1)

        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        tf = soup.select('#cphContents_cphContents_cphContents_udpContent > div.record_result > table > tbody > tr')

        for i in range(0,len(tf)):
            line = re.split('\n',tf[i].text)
            del line[0],line[-1]
            line.insert(0,year)
            record_outer.append(line)

        driver.implicitly_wait(10)
    

    return record_outer


In [ ]:
record_outer_1_1 = seleniums(url1)
record_outer_1_2 = seleniums(url2)
record_outer_2_1 = seleniums(url1)
record_outer_2_2 = seleniums(url1)

In [71]:
kb_1_1 = pd.DataFrame(record_outer_1_1)
kb_1_2 = pd.DataFrame(record_outer_1_2)
kb_2_1 = pd.DataFrame(record_outer_2_1)
kb_2_2 = pd.DataFrame(record_outer_2_2)


In [72]:
kb_1_1.to_csv('/Users/young/Desktop/kb_1_1.csv')
kb_1_2.to_csv('/Users/young/Desktop/kb_1_2.csv')
kb_2_1.to_csv('/Users/young/Desktop/kb_2_1.csv')
kb_2_2.to_csv('/Users/young/Desktop/kb_2_2.csv')


In [68]:
kb_1_1 = pd.read_csv('/Users/young/Desktop/kb_1_1.csv')
kb_1_2 = pd.read_csv('/Users/young/Desktop/kb_1_2.csv')
kb_2_1 = pd.read_csv('/Users/young/Desktop/kb_2_1.csv')
kb_2_2 = pd.read_csv('/Users/young/Desktop/kb_2_2.csv')

kb_1_1 = kb_1_1.iloc[:,1:]
kb_1_1_old = kb_1_1[kb_1_1['0']<2002]
kb_1_1_new = kb_1_1[kb_1_1['0']>=2002]
kb_1_1_new = kb_1_1_new.iloc[:,0:-2]
kb_1_2 = kb_1_2.iloc[:,1:]
kb_2_1 = kb_2_1.iloc[:,1:]
kb_2_2 = kb_2_2.iloc[:,1:]

col_kb_1_1_new = ['년도','순위','선수명','팀명','ERA','G','W','L','SV','HLD','WPCT','IP','H','HR','BB','HBP','SO','R','ER','WHIP']
col_kb_1_1_old = ['년도','순위','선수명','팀명','ERA','G','CG','SHO','W','L','SV','HLD','WPCT','TBF','IP','H','HR','BB','HBP','SO','R','ER']
col_kb_1_2 = ['년도','순위','선수명','팀명','ERA','CG','SHO','QS','BSV','TBF','NP','AVG','2B','3B','SAC','SF','IBB','WP','BK']
col_kb_2_1 = ['년도','순위','선수명','팀명','ERA','GS','Wgs','Wgr','GF','SVO','TS','GDP','GO','AO','GO/AO']
col_kb_2_2 = ['년도','순위','선수명','팀명','ERA','BABIP','P/G','P/IP','K/9','BB/9','K/BB','OBP','SLG','OPS']

kb_1_1_new.columns = col_kb_1_1_new
kb_1_1_old.columns = col_kb_1_1_old
kb_1_2.columns = col_kb_1_2
kb_2_1.columns = col_kb_2_1
kb_2_2.columns = col_kb_2_2

kb_1_1_new.reset_index(drop=True, inplace=True)

In [ ]:
# kb_1_1_new와 kb_1_2의 5번째 열부터 끝까지를 merge하여 description 변수에 저장합니다.
for i in [kb_1_2.iloc[:,5:],kb_2_1.iloc[:,5:],kb_2_2.iloc[:,5:]]:
    kb_1_1_new = kb_1_1_new.merge(i, left_index=True, right_index=True)
kb_1_1_new

In [126]:
#팀명 맞추기
new_salary = statiz_baseball_salary[statiz_baseball_salary['연도']>2001]
new_salary['팀'] = new_salary['팀'].apply(lambda x: x.upper() if x == 'kt' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: '히어로즈' if x == '우리' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: '히어로즈' if x == '키움' else x)
kb_1_1_new['팀명'] = kb_1_1_new['팀명'].apply(lambda x: 'SK' if x == 'SSG' else x)

In [127]:
# A 리스트와 B 리스트에서 중복되지 않는 요소를 찾는 함수
def find_unique_elements(A, B):
    # A 리스트에서 B 리스트에 있는 요소를 제거한 새로운 리스트 생성
    unique_elements = [elem for elem in A if elem not in B]
    return unique_elements

print(find_unique_elements(kb_1_1_new['팀명'].unique(),new_salary['팀'].unique()))
print(find_unique_elements(new_salary['팀'].unique(),kb_1_1_new['팀명'].unique()))

[]
[]


In [140]:
new_data = kb_1_1_new.merge(new_salary,left_on=['년도','선수명','팀명'],right_on=['연도','선수','팀'])
new_data = new_data.drop(['연도','팀','선수','WAR'],axis=1)
new_data

,년도,순위,선수명,팀명,ERA,G,W,L,SV,HLD,...,BABIP,P/G,P/IP,K/9,BB/9,K/BB,OBP,SLG,OPS,연봉(만원)
0,2002,2,송진우,한화,2.99,31,18,7,0,0,...,0.284,110.0,15.5,6.75,2.21,3.06,0.296,0.351,0.647,13500
1,2002,3,임창용,삼성,3.08,36,17,6,2,0,...,0.278,86.2,15.2,7.05,1.50,4.71,0.296,0.396,0.692,30000
2,2002,5,이승호,SK,3.15,27,6,12,0,0,...,0.279,84.8,16.0,9.08,2.52,3.60,0.288,0.398,0.686,8000
3,2002,8,박명환,두산,3.44,30,14,10,0,0,...,0.311,96.8,15.6,8.19,3.30,2.49,0.321,0.356,0.677,10000
4,2002,9,손민한,롯데,3.67,23,4,9,0,0,...,0.282,91.7,14.8,5.19,1.90,2.73,0.307,0.438,0.745,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,...,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,90000
286,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,...,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,50000
287,2021,17,신민혁,NC,4.41,30,9,6,0,0,...,0.307,83.0,17.2,6.64,2.73,2.43,0.327,0.420,0.747,2700
288,2021,18,임기영,KIA,4.88,28,8,8,0,0,...,0.307,90.1,16.5,7.59,2.65,2.87,0.322,0.390,0.712,7000


In [143]:
new_data.groupby('년도')['연봉(만원)'].count()

년도
2002    12
2003     9
2004    13
2005    10
2006    17
2007    13
2008    14
2009    13
2010    10
2011    10
2012    13
2013    13
2014    21
2015    20
2016    17
2017    19
2018    24
2019     5
2020    19
2021    18
Name: 연봉(만원), dtype: int64

### 2 데이터소스 컬럼명 맞추기

In [246]:
merged_df[merged_df['선수명']=='최동원']

,날짜,순위,선수명,팀명,승,패,세,경기_x,선발_x,이닝,삼진/9,볼넷/9,홈런/9,BABIP_x,LOB%_x,ERA_x,RA9-WAR_x,FIP_x,kFIP_x,WAR_x,경기_y,선발_y,완투_x,완봉_x,타자_x,안타_x,홈런_x,실점_x,자책_x,삼진_x,볼넷_x,고4_x,HBP_x,폭투_x,보크_x,PK_x,경기,선발,완투_y,완봉_y,타자_y,안타_y,홈런_y,실점_y,자책_y,삼진_y,볼넷_y,고4_y,HBP_y,폭투_y,보크_y,PK_y,삼진%,볼넷%,삼/볼,피안타율,피출루율,WHIP,BABIP_y,LOB%_y,ERA_y,RA9-WAR_y,FIP_y,kFIP_y,WAR_y
46,1983,4,최동원,롯데,9,16,4,38,29,208.7,6.38,2.20,0.73,0.297,72.50,2.89,2.45,2.29,2.53,5.43,38,29,16,1,863,202,17,89,67,148,51,2,8,3,0,0,38,29,16,1,863,202,17,89,67,148,51,2,8,3,0,0,17.1,5.9,2.90,0.257,0.302,1.21,0.297,72.50,2.89,2.45,2.29,2.53,5.43
98,1984,1,최동원,롯데,27,13,6,51,42,284.7,7.05,2.15,0.57,0.267,77.00,2.40,6.18,1.99,2.25,8.43,51,42,14,1,1132,229,18,91,76,223,68,1,14,3,0,0,51,42,14,1,1132,229,18,91,76,223,68,1,14,3,0,0,19.7,6.0,3.28,0.224,0.270,1.04,0.267,77.00,2.40,6.18,1.99,2.25,8.43
157,1985,1,최동원,롯데,20,9,8,42,35,225.0,6.44,1.64,0.28,0.257,76.00,1.92,7.33,1.56,1.79,9.43,42,35,14,4,865,170,7,60,48,161,41,2,8,3,1,0,42,35,14,4,865,170,7,60,48,161,41,2,8,3,1,0,18.6,4.7,3.93,0.214,0.250,0.94,0.257,76.00,1.92,7.33,1.56,1.79,9.43
227,1986,2,최동원,롯데,19,14,2,39,21,267.0,7.01,1.85,0.24,0.268,80.30,1.55,9.01,1.20,1.49,11.84,39,21,17,4,1039,204,7,60,46,208,55,5,6,2,0,0,39,21,17,4,1039,204,7,60,46,208,55,5,6,2,0,0,20.0,5.3,3.78,0.215,0.257,0.97,0.268,80.30,1.55,9.01,1.20,1.49,11.84
315,1987,1,최동원,롯데,14,12,2,32,25,224.0,6.55,2.45,0.24,0.318,74.20,2.81,4.78,1.96,2.24,8.14,32,25,14,4,920,218,6,80,70,163,61,2,7,3,0,0,32,25,14,4,920,218,6,80,70,163,61,2,7,3,0,0,17.7,6.6,2.67,0.262,0.311,1.25,0.318,74.20,2.81,4.78,1.96,2.24,8.14
405,1988,3,최동원,롯데,7,3,3,16,4,83.3,8.96,2.59,0.43,0.317,80.90,2.05,2.51,1.48,2.00,3.73,16,4,3,1,349,77,4,24,19,83,24,3,1,1,1,0,16,4,3,1,349,77,4,24,19,83,24,3,1,1,1,0,23.8,6.9,3.46,0.244,0.296,1.21,0.317,80.90,2.05,2.51,1.48,2.00,3.73
589,1989,95,최동원,삼성,1,2,0,8,4,30.0,2.70,5.40,0.60,0.327,82.40,2.10,0.48,4.96,4.79,-0.22,8,4,0,0,138,36,2,12,7,9,18,0,1,1,0,0,8,4,0,0,138,36,2,12,7,9,18,0,1,1,0,0,6.5,13.0,0.50,0.313,0.406,1.80,0.327,82.40,2.10,0.48,4.96,4.79,-0.22


In [242]:
merged_df.columns

Index(['날짜', '순위', '선수명', '팀명', '승', '패', '세', '경기_x', '선발_x', '이닝', '삼진/9',
       '볼넷/9', '홈런/9', 'BABIP_x', 'LOB%_x', 'ERA_x', 'RA9-WAR_x', 'FIP_x',
       'kFIP_x', 'WAR_x', '경기_y', '선발_y', '완투_x', '완봉_x', '타자_x', '안타_x',
       '홈런_x', '실점_x', '자책_x', '삼진_x', '볼넷_x', '고4_x', 'HBP_x', '폭투_x', '보크_x',
       'PK_x', '경기', '선발', '완투_y', '완봉_y', '타자_y', '안타_y', '홈런_y', '실점_y',
       '자책_y', '삼진_y', '볼넷_y', '고4_y', 'HBP_y', '폭투_y', '보크_y', 'PK_y', '삼진%',
       '볼넷%', '삼/볼', '피안타율', '피출루율', 'WHIP', 'BABIP_y', 'LOB%_y', 'ERA_y',
       'RA9-WAR_y', 'FIP_y', 'kFIP_y', 'WAR_y'],
      dtype='object')

In [243]:
new_data.columns

Index(['년도', '순위', '선수명', '팀명', 'ERA', 'G', 'W', 'L', 'SV', 'HLD', 'WPCT',
       'IP', 'H', 'HR', 'BB', 'HBP', 'SO', 'R', 'ER', 'WHIP', 'CG', 'SHO',
       'QS', 'BSV', 'TBF', 'NP', 'AVG', '2B', '3B', 'SAC', 'SF', 'IBB', 'WP',
       'BK', 'GS', 'Wgs', 'Wgr', 'GF', 'SVO', 'TS', 'GDP', 'GO', 'AO', 'GO/AO',
       'BABIP', 'P/G', 'P/IP', 'K/9', 'BB/9', 'K/BB', 'OBP', 'SLG', 'OPS',
       '연봉(만원)'],
      dtype='object')

In [244]:
new_data

,년도,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,IP,H,HR,BB,HBP,SO,R,ER,WHIP,CG,SHO,QS,BSV,TBF,NP,AVG,2B,3B,SAC,SF,IBB,WP,BK,GS,Wgs,Wgr,GF,SVO,TS,GDP,GO,AO,GO/AO,BABIP,P/G,P/IP,K/9,BB/9,K/BB,OBP,SLG,OPS,연봉(만원)
0,2002,2,송진우,한화,2.99,31,18,7,0,0,0.720,220,199.0,16,54,11,165,82,73,1.15,8,1,21,0,905,3411,0.242,36,3,14,2,2,2,1,30,18,0,8,0,0,16,299,163,1.83,0.284,110.0,15.5,6.75,2.21,3.06,0.296,0.351,0.647,13500
1,2002,3,임창용,삼성,3.08,36,17,6,2,0,0.739,204 1/3,188.0,24,34,24,160,82,70,1.09,0,0,16,0,841,3103,0.245,38,3,10,5,4,2,0,29,14,3,4,1,0,13,209,216,0.97,0.278,86.2,15.2,7.05,1.50,4.71,0.296,0.396,0.692,30000
2,2002,5,이승호,SK,3.15,27,6,12,0,0,0.333,142 2/3,120.0,18,40,5,144,55,50,1.12,1,0,13,0,582,2289,0.229,31,2,9,3,4,2,0,19,5,1,6,1,0,8,129,135,0.96,0.279,84.8,16.0,9.08,2.52,3.60,0.288,0.398,0.686,8000
3,2002,8,박명환,두산,3.44,30,14,10,0,0,0.583,185 2/3,173.0,12,68,7,169,78,71,1.30,0,0,17,0,782,2904,0.250,30,4,9,5,2,5,1,29,14,0,0,0,0,17,191,165,1.16,0.311,96.8,15.6,8.19,3.30,2.49,0.321,0.356,0.677,10000
4,2002,9,손민한,롯데,3.67,23,4,9,0,0,0.308,142 1/3,144.0,19,30,4,82,68,58,1.22,2,1,13,0,586,2109,0.266,28,4,7,4,1,5,0,22,4,0,2,0,0,24,187,132,1.42,0.282,91.7,14.8,5.19,1.90,2.73,0.307,0.438,0.745,12000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,0.455,165 2/3,162.0,12,67,7,164,85,75,1.38,0,0,14,0,714,2968,0.259,25,2,6,8,0,8,1,31,10,0,0,0,0,17,137,171,0.80,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,90000
286,2021,16,스트레일리,롯데,4.07,31,10,12,0,0,0.455,165 2/3,162.0,12,67,7,164,85,75,1.38,0,0,14,0,714,2968,0.259,25,2,6,8,0,8,1,31,10,0,0,0,0,17,137,171,0.80,0.328,95.7,17.9,8.91,3.64,2.45,0.333,0.363,0.696,50000
287,2021,17,신민혁,NC,4.41,30,9,6,0,0,0.600,145,155.0,17,44,5,107,75,71,1.37,0,0,9,0,625,2490,0.271,28,3,2,2,0,2,2,25,9,0,2,0,0,13,130,182,0.71,0.307,83.0,17.2,6.64,2.73,2.43,0.327,0.420,0.747,2700
288,2021,18,임기영,KIA,4.88,28,8,8,0,0,0.500,153,155.0,15,45,11,129,85,83,1.31,0,0,13,0,662,2522,0.262,29,1,6,8,0,6,2,28,8,0,0,0,0,5,146,170,0.86,0.307,90.1,16.5,7.59,2.65,2.87,0.322,0.390,0.712,7000
